# Week 2 Programming Assignment 

Remark: 

Please upload your solutions of this assignment to Canvas with a file named "Programming_Assignment_2 _yourname.ipynb" before 11:59pm May 30.

=================================================================================================================

### **Problem 1 (4 pt).** Use stochastic gradient descent method to train MNIST with the logisitc regression model to achieve at least 92% test accuracy. Print the results with the following format:

   "Epoch: i, Training accuracy: $a_i$, Test accuracy: $b_i$"

where $i=1,2,3,...$ means the $i$-th epoch,  $a_i$ and $b_i$ are the training accuracy and test accuracy computed at the end of $i$-th epoch.

In [1]:
# write your code for solving probelm 1 in this cell
# Hint: you can tune the number of epoches, learning rate and mini-batch size.
import torch
import torch.nn as nn # Import functions from torch.nn
import torchvision
import matplotlib.pyplot as plt
import torch.optim as optim

def model(input_size,num_classes):
    return nn.Linear(input_size,num_classes)

input_size = 784 #1*28*28=784
num_classes = 10 #10


my_model =model(input_size, num_classes)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(my_model.parameters(), lr=0.1)

MNIST_transform = torchvision.transforms.ToTensor()

trainset = torchvision.datasets.MNIST(root='./data', train= True, download=True, transform=MNIST_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=50, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train= False, download=True, transform=MNIST_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False) 
# Write a loop to train the model using the given optimizer and loss functions.
num_epochs = 20
# One epoch is when an entire dataset is passed through the neural network only once.
for epoch in range(num_epochs):
    num_train = 0
    total_tr = 0
    for i, (images, labels) in enumerate(trainloader):
        # You can check the size of image before images.reshape, which is [batch_size,1,28,28]
        #print(images.size()) 
        # Reshape MNIST images to (batch_size, input_size)
        images = images.reshape(images.size(0), 28*28)
        
        # Forward pass to get the loss
        outputs = my_model(images) # the outputs of xW^{T}+b
        loss = criterion(outputs, labels)
        
        # Backward and compute the gradient
        optimizer.zero_grad()
        loss.backward()  #backpropragation
        optimizer.step()
        
        p_max1, predicted1 = torch.max(outputs, 1) 
        total_tr += labels.size(0) # you can also use labels.size()[0], see the test  below
        num_train += (predicted1 == labels).sum()
        
    num_test = 0
    total_te = 0
    for i, (images, labels) in enumerate(testloader):
        images = images.reshape(images.size(0), 1*28*28)
        outputs = my_model(images)
        
        p_max, predicted = torch.max(outputs, 1) 
        total_te += labels.size(0) # you can also use labels.size()[0], see the test  below
        num_test += (predicted == labels).sum()
        
    print('Epoch [{}], train accuracy: {}, test accuracy: {}' .format(epoch+1,float(num_train)/total_tr,float(num_test)/total_te))
    

Epoch [1], train accuracy: 0.8810166666666667, test accuracy: 0.9105
Epoch [2], train accuracy: 0.90835, test accuracy: 0.9156
Epoch [3], train accuracy: 0.9143166666666667, test accuracy: 0.9181
Epoch [4], train accuracy: 0.9178333333333333, test accuracy: 0.9211
Epoch [5], train accuracy: 0.9195, test accuracy: 0.9211
Epoch [6], train accuracy: 0.9206166666666666, test accuracy: 0.9232
Epoch [7], train accuracy: 0.9218666666666666, test accuracy: 0.9227
Epoch [8], train accuracy: 0.9235333333333333, test accuracy: 0.9228
Epoch [9], train accuracy: 0.9235166666666667, test accuracy: 0.9226
Epoch [10], train accuracy: 0.9237666666666666, test accuracy: 0.9218
Epoch [11], train accuracy: 0.92495, test accuracy: 0.924
Epoch [12], train accuracy: 0.9259, test accuracy: 0.924
Epoch [13], train accuracy: 0.9262, test accuracy: 0.925
Epoch [14], train accuracy: 0.9260666666666667, test accuracy: 0.9227
Epoch [15], train accuracy: 0.9269666666666667, test accuracy: 0.9252
Epoch [16], train ac

=================================================================================================================

### **Problem 2 (6 pts).** Extract the subset of data which are labeled with 0,1,3,4,7 from MNIST dataset. Use both full gradient descent method and stochastic gradient descent method to train this subset with the logisitc regression model to achieve the training and test accuracy as high as possible. Print the results with the following format:

* For full gradient descent method, print:

    "Full gradient descent, Epoch: i, Training accuracy: $a_i$, Test accuracy: $b_i$"


* For stochastic gradient descent method, print:

    "Stochastic gradient descent, Epoch: i, Training accuracy: $a_i$, Test accuracy: $b_i$"

where $i=1,2,3,...$ means the $i$-th epoch,  $a_i$ and $b_i$ are the training accuracy and test accuracy computed at the end of $i$-th epoch.

In [2]:
import torch
import torch.nn as nn # Import functions from torch.nn
import torchvision
import torch.optim as optim

def model(input_size,num_classes):
    return nn.Linear(input_size,num_classes)

input_size = 784 #1*28*28=784
num_classes = 10

my_model =model(input_size, num_classes)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(my_model.parameters(), lr=0.1)

MNIST_transform = torchvision.transforms.ToTensor()

trainset = torchvision.datasets.MNIST(root='./data', train= True, download=True, transform=MNIST_transform)
testset = torchvision.datasets.MNIST(root='./data', train= False, download=True, transform=MNIST_transform)

def get_indices(dataset):
    indices =  []
    for j in [0,1,3,4,7]:
        for i in range(len(dataset.targets)):
            if (dataset.targets[i] == j):
                indices.append(i)
    return indices

idx = get_indices(trainset)
idy = get_indices(testset)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=1,sampler =torch.utils.data.sampler.SubsetRandomSampler(idx))
testloader = torch.utils.data.DataLoader(testset, batch_size=1, sampler =torch.utils.data.sampler.SubsetRandomSampler(idy))

# Write a loop to train the model using the given optimizer and loss functions.
num_epochs = 20
# One epoch is when an entire dataset is passed through the neural network only once.
def opti(epoch_size,trainloader,testloader):
    for epoch in range(epoch_size):
        num_train = 0
        total_tr = 0
        for i, (images, labels) in enumerate(trainloader):
            images = images.reshape(images.size(0), 28*28)
        
        # Forward pass to get the loss
            outputs = my_model(images) # the outputs of xW^{T}+b
            loss = criterion(outputs, labels)
        
        # Backward and compute the gradient
            optimizer.zero_grad()
            loss.backward()  #backpropragation
            optimizer.step()
        
            p_max1, predicted1 = torch.max(outputs, 1) 
            total_tr += labels.size(0) # you can also use labels.size()[0], see the test  below
            num_train += (predicted1 == labels).sum()
        
        num_test = 0
        total_te = 0
        for i, (images, labels) in enumerate(testloader):
            images = images.reshape(images.size(0), 1*28*28)
            outputs = my_model(images)
        
            p_max, predicted = torch.max(outputs, 1) 
            total_te += labels.size(0) # you can also use labels.size()[0], see the test  below
            num_test += (predicted == labels).sum()
        
        print('Epoch [{}], train accuracy: {}, test accuracy: {}' .format(epoch+1,float(num_train)/total_tr,float(num_test)/total_te))
    return('done the process')

print('For full gradient descent method:')
print(opti(num_epochs,trainloader,testloader))
print('\n')
trainloaderb = torch.utils.data.DataLoader(trainset, batch_size=100,sampler =torch.utils.data.sampler.SubsetRandomSampler(idx))
print('For stochastic gradient descent method:')
print(opti(num_epochs,trainloaderb,testloader))



For full gradient descent method:
Epoch [1], train accuracy: 0.961460052422095, test accuracy: 0.9698149951314509
Epoch [2], train accuracy: 0.9719121120926771, test accuracy: 0.9626095423563777
Epoch [3], train accuracy: 0.97430670161473, test accuracy: 0.9678675754625121
Epoch [4], train accuracy: 0.9748891693363104, test accuracy: 0.9777994157740993
Epoch [5], train accuracy: 0.9762159013687991, test accuracy: 0.9631937682570594
Epoch [6], train accuracy: 0.9767336504546484, test accuracy: 0.9777994157740993
Epoch [7], train accuracy: 0.9776073520370191, test accuracy: 0.978188899707887
Epoch [8], train accuracy: 0.9772513995404977, test accuracy: 0.9764362220058422
Epoch [9], train accuracy: 0.9790958806588357, test accuracy: 0.9762414800389484
Epoch [10], train accuracy: 0.9795165517910882, test accuracy: 0.9686465433300876
Epoch [11], train accuracy: 0.9785781315729865, test accuracy: 0.9700097370983447
Epoch [12], train accuracy: 0.9787075688444488, test accuracy: 0.978188899707

=================================================================================================================